In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

case_to_evaluate="9_3_2"

In [69]:
combined_imputed_cells = pd.read_csv(f"data/{case_to_evaluate}/vae_normal/combined_imputed_cells.csv")
combined_replaced_cells = pd.read_csv(f"data/{case_to_evaluate}/vae_normal/combined_replaced_cells.csv")
combined_test_cells = pd.read_csv(f"data/{case_to_evaluate}/vae_normal/combined_test_cells.csv")
combined_test_cells

,Mean Neighbor Intensity pERK,Mean Neighbor Intensity Rad51,Mean Neighbor Intensity CCND1,Mean Neighbor Intensity Vimentin,Mean Neighbor Intensity aSMA,Mean Neighbor Intensity Ecad,Mean Neighbor Intensity ER,Mean Neighbor Intensity PR,Mean Neighbor Intensity EGFR,Mean Neighbor Intensity pRB,...,Eccentricity,X_centroid,Y_centroid,Phenotype,# of Immune Cells,# of Neoplastic Epithelial Cells,# of Stroma Cells,Cell Neighborhood,Cell Neighborhood Encoded,Radius
0,-1.982650,-2.573164,-2.214435,-0.907921,-1.803212,-1.541311,-2.458003,-1.415121,-1.582431,-1.283895,...,0.263662,5040.107143,380.589286,Stroma (aSMA+),1,0,0,Immune,1,50
1,-2.053947,-2.408625,-2.256200,-0.959125,1.026614,-1.297823,-2.112923,-1.012705,-0.904860,-0.819339,...,-0.088532,5039.914286,390.609524,Immune,0,0,1,Stroma (aSMA+),4,50
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.465650,9388.723301,1230.747573,Basal,0,0,0,Unknown,5,50
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.826378,4977.342105,1463.052632,Immune,0,0,0,Unknown,5,50
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.058586,3591.947368,1820.506579,Immune,0,0,0,Unknown,5,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118429,-5.000000,-5.000000,-3.620415,-2.008235,-2.273108,-1.624496,-2.538128,-1.923214,-2.132538,-1.640076,...,-0.398443,12581.981651,9547.192661,Immune,1,0,0,Immune,1,30
118430,-5.000000,-5.000000,-3.586583,-1.871904,-2.258850,-1.615115,-2.477666,-1.917842,-2.179673,-1.718813,...,0.723904,12553.467354,9548.766323,Immune,1,0,0,Immune,1,30
118431,-4.708312,-5.000000,-3.599913,-1.886476,-2.290937,-1.637294,-2.532161,-1.952563,-2.178062,-1.736037,...,-0.381631,9658.215385,9636.630769,Immune,1,0,0,Immune,1,30
118432,-4.734637,-5.000000,-3.586597,-1.863037,-2.311429,-1.639740,-2.533249,-1.876359,-2.181547,-1.707427,...,0.089627,9662.470930,9648.145349,Immune,1,0,0,Immune,1,30


# Index Replacements

In [70]:
index_replacements = pd.read_csv(f"data/{case_to_evaluate}/index_replacements.csv")
#index_replacements
index_replacements = index_replacements.T.to_dict()

# Features to im

In [71]:
features = pd.read_csv(f"data/{case_to_evaluate}/vae_normal/features_to_impute.csv")
features = features["0"].to_list()
features

FileNotFoundError: [Errno 2] No such file or directory: 'data/9_3_2/vae_normal/features_to_impute.csv'

# R2 calculation

In [ ]:
from typing import List, Dict
from sklearn.metrics import r2_score

def evaluate_performance(features: List, index_replacements: Dict, test_data: pd.DataFrame,
                             imputed_data: pd.DataFrame):
        """
        Evaluates the performance per feature
        @param features:
        @param index_replacements:
        @param test_data:
        @param imputed_data:
        @return: Returns a dataframe with r2 scores for performance evaluation
        """
        score_data: List = []

        for feature in features:
            if "X_centroid" in feature or "Y_centroid" in feature or "Origin" in feature:
                continue

            # Store all cell indexes, to be able to select the correct cells later for r2 comparison
            cell_indexes_to_compare: list = []
            for key, replaced_features in index_replacements.items():
                key = int(key)
                replaced_features = list(replaced_features.values())
                
                if feature in replaced_features:
                    cell_indexes_to_compare.append(key)
           
            score = r2_score(test_data[feature].loc[cell_indexes_to_compare],
                                  imputed_data[feature].loc[cell_indexes_to_compare])
                    
            score_data.append({
                "Feature": feature,
                "Score": score if score >= 0 else 0
            })

        return pd.DataFrame().from_records(score_data)

In [ ]:
combined_r2_scores_data = []
for radius in combined_test_cells["Radius"].unique():
    selected_test_cells = combined_test_cells[combined_test_cells["Radius"] == radius].reset_index(drop=True)
    selected_imputed_cells = combined_imputed_cells[combined_imputed_cells["Radius"] == radius].reset_index(drop=True)
    r2_scores = evaluate_performance(features, index_replacements, selected_test_cells, selected_imputed_cells)
    r2_scores["Radius"] = radius
    combined_r2_scores_data.append(r2_scores)
    
combined_r2_scores = pd.concat(combined_r2_scores_data)
combined_r2_scores.to_csv(f"data/{case_to_evaluate}/vae_normal/combined_r2_scores.csv", index=False)
combined_r2_scores

In [ ]:
fig = plt.figure()
sns.barplot(x="Feature", y="Score",hue="Radius", data=combined_r2_scores)
for ax in fig.axes:
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)
fig.set_size_inches(30,7)
plt.title("VAE Imputation R2 Score") 
plt.tight_layout()
plt.savefig(f'data/{case_to_evaluate}/vae_normal/combined_r2_scores.png', dpi=fig.dpi,bbox_inches='tight',facecolor='w')
# plt.show()
